#### Chris Sutton
#### 
#### Lab#4¶

In [1]:
# In this lab I engineered features using multiple methods. For problems B & D, I analyised the training data and chose 
# a path of creating discriminating unigram features. I reviewed the occurence of terms in the positive and negative class
# and chose to pick features that occur more frequently in one class or another. The features were built and passed to the 
# sklearn countVectorizer which, in the case of problems A and D, simply vectorized them. My total number of features in
# problems A and D was 200. For problem E chose to use sklearn to read my files directly, bypassing my custom unigram
# features to build unigram and bigrams for a total of 500 features. For classifiers I chose to use Naive Bayes (problem A) 
# and a support vector classifier (problems D,E). While 200 and 500 features is a lot for data consisting of 2k samples, a
# lower number of features results in worse DEVset performance. My number of features is targeting ~.80 F1score. This was
# an interesting lab, consume reviews and returning sentiment simply based on the two classes of text.  
 






##### (a) Study the training data. Examine train.tsv. Do you see indications of positive or negative sentiment (e.g., words like
##### 'good' or 'terrible'). Are reviews balanced or polarized?
##### 4 points. Identify some useful features for positive or negative sentiment. Give 10 examples and the relative
##### frequency you observe in both classes (e.g., 'good', 33.4% (pos), 7.6% (neg).)
##### 2 points. Report any other observations about the data that are helpful for this task.

In [2]:
# I processed the data and analyized it to find a difference in the frequency of occurence depending on if they 
# appeared in the positive or negative class. Below are some features found by taking thr delta of occurence between
# classes. I choose 5 that skewed toward the positive class an 5 that skewed toward the negaitve class. While only
# 10 features are shown below I used many more in building the sentiment classifier. 

# One interesting feature to note is
# that the exclaimation mark certainly skew toward the positive class and shows that the positive reviewers tend to use 
# then often.

In [3]:
# interesting features are: 
# ! (.50 pos, .28 neg)
# always (.22 pos, .1 neg)
# good (.54 pos, .42 neg)
# great (.35 pos, .17 neg)
# delicious (.14 pos,.03 neg)
# n't (.45 pos, .59 neg)
# better (.12 pos, .22 neg)
# ordered (.14 pos, .23 neg)
# bad (.07 pos, .16 neg)
# like (.32 pos, .40 neg)


In [4]:
import re
import collections
import pandas
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = set(stopwords.words('english'))
stop.update(['The','.',',','I','If'])
# nltk.=ownload('stopwords')

In [5]:
docidTrain= re.findall(r"\t(\w+)\t",open('train.tsv','rt',encoding='utf-8').read(), re.MULTILINE)
ratingTrain= [int(x) for x in re.findall(r"^\d",open('train.tsv','rt',encoding='utf-8').read(), re.MULTILINE)]
strReview= re.findall(r"\t.*\t(.*\n)",open('train.tsv','rt',encoding='utf-8').read(), re.MULTILINE)

dociddev= re.findall(r"\t(\w+)\t",open('dev.tsv','rt',encoding='utf-8').read(), re.MULTILINE)
ratingdev= [int(x) for x in re.findall(r"^\d",open('dev.tsv','rt',encoding='utf-8').read(), re.MULTILINE)]
strdev= re.findall(r"\t.*\t(.*\n)",open('dev.tsv','rt',encoding='utf-8').read(), re.MULTILINE)

docidTest= re.findall(r"\t(\w+)\t",open('test.tsv','rt',encoding='utf-8').read(), re.MULTILINE)
ratingTest= [int(x) for x in re.findall(r"^\d",open('test.tsv','rt',encoding='utf-8').read(), re.MULTILINE)]
strTest= re.findall(r"\t.*\t(.*\n)",open('test.tsv','rt',encoding='utf-8').read(), re.MULTILINE)

In [6]:
docidTrain[0]

'z8DDztUxuIoHYHddDL9zQ'

In [7]:
def removeStop(strReview_):
    # remove stop words -----------------------------------------------
    review=[]
    reviewCount=[]
    for r in strReview_:
        txt=[]
    #     reviewCount.append(collections.Counter(word_tokenize(r)))
        for word in word_tokenize(r):
            if word not in stop:
                txt.append(word)
            else:
                pass
        reviewCount.append(collections.Counter(txt)) #count of term per review for analysis
        review.append(txt) # list of string term per review
    return review, reviewCount

In [8]:
def makeFeatures(reviewCount_,ratingTrain_, posDelta_, negDelta_):
# split data into good versus bad DF terms dict------------------------------------
    dfg={} # df of good review terms 
    dfb={} # df of bad review terms
    for r, ra in zip(reviewCount_, ratingTrain_):
        dfc={}
        for k in r.keys():
            if k not in dfc.keys(): 
                dfc[k]=1 # add 1 for each unique word in current reveiw
            else:
                pass
        if ra ==4:
            for ke in dfc.keys(): # build dict of good reviews
                if ke not in dfg.keys():
                    dfg[ke]=1
                else:
                    dfg[ke]+=1
        else:
            for ke in dfc.keys(): #build dict of bad reviews
                if ke not in dfb.keys():
                    dfb[ke]=1
                else:
                    dfb[ke]+=1    

# find all common keys in good verus bad reviews and remove non-common keys----------------------
    c=dfg.keys() ^ dfb.keys()
    for key in c:
        if key in dfg.keys():
            del dfg[key]
        else:
            del dfb[key]
# make features by take the diffence of the difference in occurence between classes----------------------------
    features={}
    for k in dfg:
        if dfg[k]-dfb[k]>posDelta_: #45 #10 #5
            features[k]=dfg[k]-dfb[k]
    for k in dfg:
        if dfb[k]-dfg[k]>negDelta_: #56 #10 #5
            features[k]=dfb[k]-dfg[k]
    return features

In [9]:
def stopOutAllOtherWords(review_, features_):
# stop out all other word in documents that are not features--------------------------
    out=[]
    for r in review_:
        l=''
        for w in r:
            if w in features_.keys():
                l+=w+' '
        out.append(l) # list of txt output to scikit
    return out

#### (b) Train a classifier. Using the training partition build a supervised model using a learning model of your choice. We
#### suggest a simple bag of words model as a baseline feature representation. Then make predictions for the dev and test
#### partitions and write those to a file.
#### 4 points. Describe your approach. Include details such as the algorithm used, important parameters, the type
#### of features, and the total number of features.
#### 2 points. Print out a feature representation for the first document in the dev set. (Skip 'zeros'.)
#### 4 points. Print docid [tab] prediction for the first 10 documents in the dev file. Prediction should be 2 or 4

In [10]:
# My approach was to study the data separated by by review and also by class. I counted the DF of terms per review and 
# summed them per class to understand the imbalance of review terms per class. next I ensured that both classes contained
# the same vocabulary by taking the semetric differnece betwen the two classes. From the positive and negative classes
# I built feature by looking for descriminating differences between the classed DF of the remaining terms. I selected all
# terms with a between class DF greater then 5. These features are then processed by sklearn countVectorizer into an matrix
# to the ML models can process. sklearn counter vectorizer does remove some of my intended features. The documentation 
# states that all puncuation is removed with vectorizing. The resulting number of features is 862. The number of features 
# directly affects performance of the classifier. 
# 
# 


In [11]:
# call functions to save the variables for later model buildingearn---------------------------------
review ,reviewCount=removeStop(strReview)
features= makeFeatures(reviewCount,ratingTrain, 20, 20)
out= stopOutAllOtherWords(review, features)

In [12]:
# feed my selected features into sklearn to vectorize the features
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(out)
X_train_counts.shape
# X_train_counts[0]

(2000, 200)

In [13]:
# call sklearn MNNB while iputting class priors [.5, .5]
from sklearn.naive_bayes import MultinomialNB
MNNB= MultinomialNB(class_prior=[.5,.5]).fit(X_train_counts, ratingTrain)

In [14]:
# process the dev.tsv document in a similar manner so that it will feed into sklearn.
# The first document in the dev set is printed and showing only used features
reviewDev ,reviewCountDev=removeStop(strdev)
outDev= stopOutAllOtherWords(reviewDev, features)
count_vectDev = CountVectorizer()
XDev = count_vect.transform(outDev)
print(XDev.shape)
print(outDev[0])
# print(XDev.toarray()[0])

(2000, 200)
restaurant ) place good enjoyed Chicken good tasty trying menu Great menu told friendly wanted menu 


In [15]:
# make prediction on the dev document (first 10 documents)
predicted = MNNB.predict(XDev)
count=0
for i in range(10):
       print(dociddev[i],"\t",predicted[i])
    
# print(count)


ZSJnW6faaNFQoqq4ALqYg 	 4
Rcbv11hm5AYEwZyqYwAvg 	 4
rkRTjhu5szaBggeFVcVJlA 	 4
dhmeDsQGUS1FXMLs49SWjQ 	 4
z9zfIMYmRRCE4ggfOIieEw 	 4
Xtb3pGSh39bqcozkBECw 	 2
DOUflAGzxLsXG6xOmR1w 	 2
0RxCEWURe08CTcZt95F4AQ 	 4
MzUg5twEcCyd0X6lBMP2Lg 	 2
uNlw2D5CYKk0wjNxLtYw 	 4


#### (c) Evaluate your predictions. Using your predictions from part (b), compute precision, recall and F1 scores for just the
#### positive class over the full dev set. Show the work in your computation (i.e., the numerators and denominators for
#### precision and recall). Recall is the percentage of positive predictions (i.e., 4 stars) in the test file that are correctly
#### predicted to belong to the positive class. Precision is the percentage of positive predictions in the predictions file which
#### are indeed correct according to the test file labels. For reference: F1 = 2*P*R/(P+R).
#### 4 points. Calculate and report Precision, Recall, and F1 scores.
#### 4 points. Reasonable results (e.g., F1 score > 65). Around 80% is probably not very hard to attain.
#### 4 points. Find a few mistakes that the classifier makes. Present a couple of incorrect predictions that you find
#### interesting along with a short comment.

In [16]:
# (3rd doc) truth=2, predicted=4, features= "used favorite family recently cut back menu many favorites gone menu online see 's left"
# comment- I think the issue here is the use of favorites and favorite weighting the classifier into a positive review

# (10th doc) truth=2, predicted=4, features ="lunch day little Chinese restaurant also style menu Not many dishes found n't really veggies
# dish 2 star" 
# comment- beacuse countVectorizer removed all punctuation the (n't), which is frequently used in negative reviews, was
# removed which changed the resulting classification 

# (29th doc) truth=4, predicted=2 "thought would give $ box try went tonight took go busy obviously $ box got breast
# spicy sides beans potato ( : ( ) ... included honest saying far juicy breast crispy beans well You also option 3 
# instead 2 piece They busy drive thru No one rude busy After experience location $ meal full place clean : 
# tables people experience staff gone There sign window"
# comment - I think the number of features appearing in this review coupled with the use of negative words has an effect
# on the prediction 
# 

In [17]:
# evaluation of precision, recall, F1score of custom features and a MNNB model
A=0
B=0
C=0
count=0
for truth, p in zip(ratingdev, predicted,):
   
#     print(count,truth,p)
    count+=1
    if truth==4 and p==4:
        A+=1
    elif truth==2 and p==4:
        B+=1
    elif truth==4 and p==2:
        C+=1
Precision=A/(A+B)
Recall=A/(A+C)
print('Precision : ''{: .3f}'.format(A/(A+B)))
print('   Recall : ''{: .3f}'.format(A/(A+C)))
print('  F1Score : ''{: .3f}'.format(2*Precision*Recall/(Precision+Recall)))


Precision :  0.796
   Recall :  0.810
  F1Score :  0.803


#### (d) Build a second classifier. Repeat steps (b) and (c) using a different machine learning algorithm2.
#### (2 points. Provide the same information requested for step (b)
#### (2 points. Evaluate and provide the same information from step (c)
#### (2 points. Briefly compare your results between the two classifiers.

In [18]:
# # My approach followed the same methology from above. The feature are also processed by sklearn countVectorizer into a
# matrix to the ML models can process. The resulting number of features is 862. Similarly to the NaiveBayes, the number
# of features directly affects performance of the classifier. Additionally,the Regularization parameter (C) was chosen to 
# be .011 by search to achieve the highest precision, recall and F1score.
# 
# (8th doc) truth=4, SVC predicted 2, MNNB predicted 4, features="Our Vegas friend drinks late ( ) All needed hear small 
# bar area place open stopped half priced ( happy ) friend 's friend cool guy working n't want going ordered dishes steak
# cheese bread Both good cheese bread really good also one one ' decent got 'd get without creamy tomato sauce 'd say 
# food 3 stars 's location solid 4 stars "
# comment- nothing really stands out to me as what caused the mis-classifiaction. It could be the words "late","ordered"

# (113th doc) truth=2, SVC predicted 4, MNNB predicted 2, features="Not good definitely better big chain fast food 
# fries flavor sweet ( ? ) burger `` meh '' n't go back need frozen high 
# comment- The missing n't due to processign of the countVectorizer may have made a different here. however, the MNNB model
# still predicted the correct class. some predictive features for the negative class such as "better" appear here but it 
# didn't seem to help the SVC decision boundary.


In [19]:
from sklearn.svm import LinearSVC

In [20]:
svc =LinearSVC( C=.012).fit(X_train_counts, ratingTrain)

In [21]:
psvc=svc.predict(XDev)

In [22]:
# evaluation of precision, recall, F1score of custom features and a linearSupportVectorClassifier model
A=0
B=0
C=0
counter=0
for truth, p,pr in zip(ratingdev, psvc,predicted ):
#     print(counter,truth,p,pr)
    counter+=1
    if truth==4 and p==4:
        A+=1
    elif truth==2 and p==4:
        B+=1
    elif truth==4 and p==2:
        C+=1
Precision=A/(A+B)
Recall=A/(A+C)
print('Precision : ''{: .3f}'.format(A/(A+B)))
print('   Recall : ''{: .3f}'.format(A/(A+C)))
print('  F1Score : ''{: .3f}'.format(2*Precision*Recall/(Precision+Recall)))

Precision :  0.799
   Recall :  0.792
  F1Score :  0.796


#### (e) Feature engineering. For one of your classifiers explore using additional features beyond bags of words to improve
#### performance. For example, you could use an English sentiment dictionary such as SentiWordNet which is included with
#### the lab data. This file requires some pre-processing to extract sentiment terms; I’ve heard that SentiWordNet might be
#### available in NLTK. Some other ideas would be to look at word bigrams (e.g., “delicious food”, “horrible service”),
#### handling negation (see J&M chapter 4), use of punctuation (!!!!), or emoticons :-).
#### 3 points. Describe any features you added and the effect of using them on the dev set.
#### 3 points. In Canvas submit predictions for the test.tsv file using what you think is your best model. The file
#### should be named YOURJHED.txt. The format should be docid [tab] prediction. Prediction should be either
#### 2 or 4

In [23]:
# For part E I chose to bypass my custom processing used earlier that selected only discriminating terms between classes. 
# Here I chose to consume the train and dev, test test directly and allow countVectorizer to provide stop words, single
# and bigrams of up to 500 features, with a Regularization parameter (C) of .009. I wanted to test the direct performance 
# of sklearn over my custom term feature generation.It turns out this implementation provides a even balance between 
# precision and recall and likely we generalize well to test data. 
# 


In [24]:
count_vectPartE = CountVectorizer(stop_words='english',ngram_range=(1, 2),max_features=500)
X_train_countsPartE = count_vectPartE.fit_transform(strReview)
# max_df=.9
XDevPartE = count_vectPartE.transform(strdev)
print(XDevPartE.shape)
svcparte =LinearSVC( C=.009).fit(X_train_countsPartE, ratingTrain)
psvcPartE=svcparte.predict(XDevPartE)
# print(count_vectPartE.vocabulary_)

(2000, 500)


In [25]:
# evaluation of precision, recall, F1score of up to 10K single and bigrams with a linearSupportVectorClassifier model
A=0
B=0
C=0
counter=0
for truth, p in zip(ratingdev, psvcPartE ):
#     print(counter,truth,p)
    counter+=1
    if truth==4 and p==4:
        A+=1
    elif truth==2 and p==4:
        B+=1
    elif truth==4 and p==2:
        C+=1
Precision=A/(A+B)
Recall=A/(A+C)
print('Precision : ''{: .3f}'.format(A/(A+B)))
print('   Recall : ''{: .3f}'.format(A/(A+C)))
print('  F1Score : ''{: .3f}'.format(2*Precision*Recall/(Precision+Recall)))

Precision :  0.805
   Recall :  0.797
  F1Score :  0.801


In [26]:
count_vectPartE = CountVectorizer(stop_words='english',ngram_range=(1, 2),max_features=500)
X_train_countsPartE = count_vectPartE.fit_transform(strReview)
# # max_df=.9
XTestPartE = count_vectPartE.transform(strTest)

svcparte =LinearSVC( C=.009).fit(X_train_countsPartE, ratingTrain)
psvcPartE=svcparte.predict(XTestPartE)
# print(count_vectPartE.vocabulary_)

In [27]:
# print file docid tab prediction
file1 = open("CSUTTO16.txt", "w") 
for doc, pred in zip(docidTest,psvcPartE) :
    file1.write(str(doc)+'\t'+str(pred)+'\n')
file1.close()
